In [87]:
import pandas as pd
import numpy as np

# Read in data
metered = pd.read_csv('hrl_load_metered - 20170201-20200131.csv')
temp = pd.read_csv('hr_temp_20170201-20200131_subset.csv')

In [88]:
# Convert date to datetime64 and sort
metered['datetime_beginning_ept'] = pd.to_datetime(metered['datetime_beginning_ept'], format='%m/%d/%Y %H:%M')
metered.rename(columns={'datetime_beginning_ept': 'date'}, inplace=True)
metered.head(5)

,datetime_beginning_utc,date,nerc_region,mkt_region,zone,load_area,mw,is_verified
0,2/1/2017 5:00,2017-02-01 00:00:00,RFC,WEST,DUQ,DUQ,1419.881,True
1,2/1/2017 6:00,2017-02-01 01:00:00,RFC,WEST,DUQ,DUQ,1379.505,True
2,2/1/2017 7:00,2017-02-01 02:00:00,RFC,WEST,DUQ,DUQ,1366.106,True
3,2/1/2017 8:00,2017-02-01 03:00:00,RFC,WEST,DUQ,DUQ,1364.453,True
4,2/1/2017 9:00,2017-02-01 04:00:00,RFC,WEST,DUQ,DUQ,1391.265,True


In [89]:
# convert date to datetime64 and sort
temp['DATE'] = pd.to_datetime(temp['DATE'], format='%Y-%m-%dT%H:%M:%S')
temp['DATE'] = temp['DATE'].dt.floor('h')
temp = temp.sort_values(by='DATE')
temp.rename(columns={'DATE': 'date', 'HourlyDryBulbTemperature': 'temp'}, inplace=True)
temp.head(5)

,STATION,date,REPORT_TYPE,SOURCE,temp
0,72520514762,2017-02-01 00:00:00,FM-15,7,37.0
1,72520514762,2017-02-01 01:00:00,FM-15,7,37.0
2,72520514762,2017-02-01 02:00:00,FM-15,7,36.0
3,72520514762,2017-02-01 03:00:00,FM-15,7,36.0
4,72520514762,2017-02-01 04:00:00,FM-15,7,36.0


In [90]:
# Join data and create dataset with only values we want
columns_to_keep = ['date', 'temp', 'mw']
energy_data = temp.merge(metered, how = 'left', on = 'date')[columns_to_keep]
energy_data.head(5)

,date,temp,mw
0,2017-02-01 00:00:00,37.0,1419.881
1,2017-02-01 01:00:00,37.0,1379.505
2,2017-02-01 02:00:00,36.0,1366.106
3,2017-02-01 03:00:00,36.0,1364.453
4,2017-02-01 04:00:00,36.0,1391.265


In [93]:
# Add more columns about dates that can be used to gain insights about energy data
energy_data['year'] = energy_data['date'].dt.year
energy_data['month'] = energy_data['date'].dt.month
energy_data['hour'] = energy_data['date'].dt.hour
energy_data['weekday'] = energy_data['date'].dt.weekday
energy_data.head(5)

,date,temp,mw,year,month,hour,weekday
0,2017-02-01 00:00:00,37.0,1419.881,2017,2,0,2
1,2017-02-01 01:00:00,37.0,1379.505,2017,2,1,2
2,2017-02-01 02:00:00,36.0,1366.106,2017,2,2,2
3,2017-02-01 03:00:00,36.0,1364.453,2017,2,3,2
4,2017-02-01 04:00:00,36.0,1391.265,2017,2,4,2


In [95]:
# Reorder columns to match order expected by liveProject
energy_data = energy_data[['date', 'mw', 'temp', 'hour', 'weekday', 'month', 'year']]

# Use the DATE as the index
energy_data = energy_data.set_index('date')
energy_data.head(5)

,mw,temp,hour,weekday,month,year
date,,,,,,
2017-02-01 00:00:00,1419.881,37.0,0,2,2,2017
2017-02-01 01:00:00,1379.505,37.0,1,2,2,2017
2017-02-01 02:00:00,1366.106,36.0,2,2,2,2017
2017-02-01 03:00:00,1364.453,36.0,3,2,2,2017
2017-02-01 04:00:00,1391.265,36.0,4,2,2,2017


In [103]:
energy_data[energy_data['temp'].isnull()]
energy_data['temp'] = energy_data['temp'].interpolate(method='linear')
energy_data[energy_data['temp'].isnull()]

,mw,temp,hour,weekday,month,year
date,,,,,,


In [104]:
# Split up data into training and testing data sets
t = pd.to_datetime("2020-01-01")
train = energy_data[energy_data.index < t]
test = energy_data[energy_data.index >= t]
[train, test]

[                           mw  temp  hour  weekday  month  year
 date                                                           
 2017-02-01 00:00:00  1419.881  37.0     0        2      2  2017
 2017-02-01 01:00:00  1379.505  37.0     1        2      2  2017
 2017-02-01 02:00:00  1366.106  36.0     2        2      2  2017
 2017-02-01 03:00:00  1364.453  36.0     3        2      2  2017
 2017-02-01 04:00:00  1391.265  36.0     4        2      2  2017
 ...                       ...   ...   ...      ...    ...   ...
 2019-12-31 19:00:00  1606.134  32.0    19        1     12  2019
 2019-12-31 20:00:00  1565.014  32.0    20        1     12  2019
 2019-12-31 21:00:00  1515.905  32.0    21        1     12  2019
 2019-12-31 22:00:00  1474.519  30.0    22        1     12  2019
 2019-12-31 23:00:00  1421.717  30.0    23        1     12  2019
 
 [25539 rows x 6 columns],
                            mw  temp  hour  weekday  month  year
 date                                                        